# Using Map Projections: `useCRS="provided"`

Normally Lets-Plot-Kotlin forces to use `Spatialdataset` with WGS84 CRS and throws an exception if any other CRS was used.

Value "provided" tells Lets-Plot-Kotlin that the input `SpatialDataset` already contains coordinates in the desired CRS and Lets-Plot-Kotlin should render data as is.

Let's see how can we still plot a map in the **Conus Albers** projection in Lets-Plot-Kotlin.

0. [Data preparation.](#0.-Data-preparation.)
1. [Default presentation.](#1.-Default-Presentation.)
2. [Drawing map without useCRS.](#2.-Drawing-map-without-useCRS.)  
3. [Drawing map with useCRS=provided.](#3.-Drawing-map-with-useCRS=provided.)

In [1]:
%useLatestDescriptors
%use lets-plot
// Initialize Lets-Plot GeoTools extension. 
%use lets-plot-gt(gt="[23,)")

@file:DependsOn("org.geotools:gt-epsg-hsql:[23,)")

In [2]:
import org.geotools.feature.DefaultFeatureCollection
import org.geotools.feature.simple.SimpleFeatureBuilder
import org.geotools.feature.simple.SimpleFeatureTypeBuilder
import org.geotools.geometry.jts.JTS
import org.geotools.referencing.CRS

import org.jetbrains.letsPlot.spatial.SpatialDataset

import org.locationtech.jts.geom.Coordinate
import org.locationtech.jts.geom.GeometryFactory
import org.locationtech.jts.geom.Polygon

val epsg4326 = CRS.decode("EPSG:4326", true)
val epsg5070 = CRS.decode("EPSG:5070", true)
val transform = CRS.findMathTransform(epsg4326, epsg5070, true)
val geomFactory = GeometryFactory()

#### 0. Data preparation.
For the sake of simplicity in this demo we will use Contiguous US geometry specified directly in code.

In [3]:
fun conusUS(albers: Boolean = false): SpatialDataset {
    val contiguousUS = listOf(
        (-124.6805763 to 48.3569334),
        (-123.8016701 to 45.2292060),
        (-124.4608498 to 42.6993428),
        (-124.1092873 to 39.5556772),
        (-120.3739357 to 34.6521325),
        (-117.2977638 to 32.4550250),
        (-114.7049904 to 32.3808313),
        (-111.0575294 to 31.1854903),
        (-108.0692482 to 31.2230778),
        (-103.2352638 to 28.9802130),
        (-101.5653419 to 29.5170067),
        (-97.0389748 to 25.7018660),
        (-96.5555763 to 27.7816829),
        (-95.0174904 to 28.5950728),
        (-93.1717873 to 29.1722482),
        (-89.5682716 to 28.7492996),
        (-88.6454201 to 29.4404945),
        (-86.0526466 to 29.7461959),
        (-84.0751076 to 29.5170067),
        (-82.1415138 to 25.9392133),
        (-81.0428810 to 24.5480601),
        (-80.0760841 to 25.1462170),
        (-79.9881935 to 27.0796086),
        (-80.6913185 to 28.6722146),
        (-81.1307716 to 30.2030038),
        (-80.5155373 to 31.5981298),
        (-79.6805763 to 32.4550250),
        (-78.2743263 to 33.4139619),
        (-76.5604591 to 34.3261425),
        (-75.1102638 to 35.1926087),
        (-75.4618263 to 36.7573155),
        (-75.0223732 to 37.6324504),
        (-74.1434669 to 39.2500684),
        (-73.3524513 to 39.9610698),
        (-69.7928810 to 41.0635627),
        (-69.7489357 to 41.8539636),
        (-70.0565529 to 43.1818981),
        (-67.1561623 to 44.1986975),
        (-67.2001076 to 44.8565989),
        (-68.0350685 to 47.2351890),
        (-69.3973732 to 47.1455980),
        (-70.4520607 to 45.9671404),
        (-71.1112404 to 45.1982481),
        (-73.1327248 to 45.0121468),
        (-74.7147560 to 44.9810709),
        (-76.8680763 to 43.6606692),
        (-77.8348732 to 43.5333671),
        (-79.1971779 to 42.4404343),
        (-80.7352638 to 41.9520859),
        (-82.8446388 to 41.7228993),
        (-82.0536232 to 43.6606692),
        (-82.3612404 to 45.2292060),
        (-84.0751076 to 46.3931216),
        (-86.4481544 to 47.5920411),
        (-88.2059669 to 48.2984977),
        (-90.5790138 to 48.1521153),
        (-92.8202248 to 48.4736041),
        (-95.1053810 to 48.9953118),
        (-96.9950294 to 48.9087364),
        (-99.1044044 to 48.9376115),
        (-101.3016701 to 48.8798445),
        (-102.9276466 to 48.8798445),
        (-104.5096779 to 48.9087364),
        (-105.8719826 to 48.9376115),
        (-107.4979591 to 48.9376115),
        (-108.7284279 to 48.8798445),
        (-110.3983498 to 48.8798445),
        (-111.8045998 to 48.8509359),
        (-113.0350685 to 48.8798445),
        (-114.3973732 to 48.9087364),
        (-115.7157326 to 48.8798445),
        (-116.8143654 to 48.8798445),
        (-118.3085060 to 48.9664700),
        (-119.4950294 to 48.9664700),
        (-120.8133888 to 48.8798445),
        (-121.8680763 to 48.9376115),
        (-122.6590919 to 48.9664700),
        (-123.0545998 to 48.9664700),
        (-123.3622169 to 48.5318391),
        (-124.6805763 to 48.3569334)
    ).map {(x, y) -> geomFactory.createPoint(Coordinate(x, y)) }

    val coordList: List<Coordinate> = contiguousUS.map { point ->
        if (albers) {
            JTS.transform(point, transform).coordinate
        } else {
            point.coordinate 
        }
    }
    val pg = geomFactory.createPolygon(coordList.toTypedArray())

    val featureTypeBuilder = SimpleFeatureTypeBuilder()
    featureTypeBuilder.name = "Contiguous US"
    featureTypeBuilder.crs = if (albers) epsg5070 else epsg4326
    featureTypeBuilder.add("geom", Polygon::class.java)

    val featureType = featureTypeBuilder.buildFeatureType()
    val featureBuilder = SimpleFeatureBuilder(featureType)
    featureBuilder.add(pg)

    val featureCollection = DefaultFeatureCollection()
    featureCollection.add(featureBuilder.buildFeature(null))
    return featureCollection.toSpatialDataset()
}

#### 1. Default Presentation.
This is how map rendered by default - internally Lets-Plot-Kotlin transforms WGS84 to Mercator but keep degrees as axis domains:

In [4]:
letsPlot() + geomMap(map = conusUS(), size = 0.7)

#### 2. Drawing map without useCRS.
Now we want to use geometry data in **Conus Albers** projection to draw it on a map. By default Lets-Plot-Kotlin requires input data to use WGS84 and throws exception if requirment is not satisfied:

In [5]:
letsPlot() + geomMap(map = conusUS(albers=true), size = 0.7)

The problem is found in one of the loaded libraries: check library renderers
org.jetbrains.kotlinx.jupyter.exceptions.ReplEvalRuntimeException: Geometry must use WGS84 coordinate reference system but was: NAD83 / Conus Albers.
org.jetbrains.kotlinx.jupyter.exceptions.ReplLibraryException: The problem is found in one of the loaded libraries: check library renderers
	at org.jetbrains.kotlinx.jupyter.exceptions.ReplLibraryExceptionKt.rethrowAsLibraryException(ReplLibraryException.kt:30)
	at org.jetbrains.kotlinx.jupyter.codegen.RenderersProcessorImpl.renderResult(RenderersProcessorImpl.kt:23)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1$rendered$1$1.invoke(repl.kt:464)
	at org.jetbrains.kotlinx.jupyter.config.LoggingKt.catchAll(logging.kt:41)
	at org.jetbrains.kotlinx.jupyter.config.LoggingKt.catchAll$default(logging.kt:40)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:463)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke

null

#### 3. Drawing map with useCRS=provided.
To pass this limitation set parameter `useCRS="provided"`. Now Contiguous US in Albers projection will be drawn.

In [6]:
letsPlot() + geomMap(map = conusUS(albers=true), size = 0.7, useCRS = "provided")